In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
'''Dynamic Programming, FCFS Simulated result'''
dp = 61135
fcfs = 45564

In [4]:
'''Establish an aircraft'''
class aircraft:

    # Initialize aircraft
    def __init__(self):
        self.seat_capacity = 100
        self.seat_type = ['Y', 'M', 'K']
        self.seat_price = {'f': 0, 'Y': 800, 'M': 500, 'K': 450}

In [5]:
''' Demand model from Balaiyan et al.'''
class demandmodel:
    
    # Initialize demand model
    def __init__(self):
        
        # inherent attributes from aircraft class
        self.aircraft = aircraft()
        self.seat_set = self.aircraft.seat_type
        self.seat_price = self.aircraft.seat_price
        
        # demand model parameters
        self.total_booking = 105
        self.market_share = 0.25
        self.gamma = 0.08426
        self.alpha = 0.001251
        self.beta = {'DFARE':-0.006, 'LOT3':-0.944}
        self.a = {'Y':{'DFARE':800, 'LOT3':1},
                  'M':{'DFARE':500, 'LOT3':1},
                  'K':{'DFARE':450, 'LOT3':1},
                 }
        self.fare_diff_avg = sum(self.seat_price.values()) / len(self.seat_price)
        for seat_type in self.a:
            self.a[seat_type]['DFARE'] = round(self.a[seat_type]['DFARE'] - self.fare_diff_avg, 2)

    # Calculate dm
    def dm(self):
        dm = self.total_booking/self.market_share
        return dm

    # Calcilate booking curve
    def booking_curve(self, RD2, RD1):
        booking_curve = math.exp(-self.gamma*RD2)-math.exp(-self.gamma*RD1)
        return booking_curve

    # find pj+1
    def find_pj1(self, seat):
        smaller_keys = [key for key in self.seat_price.keys() if key < seat]
        if smaller_keys:
            max_smaller_key = max(smaller_keys)
            pj1 = self.seat_price[max_smaller_key]
        else:
            pj1 = min(self.seat_price.values())
        return pj1

    # Multinomial logit model
    def mnl(self):
        choose_prob = {}
        value_dict = {}
        for seat in self.seat_set:
            result = {key: self.beta[key] * value for key, value in self.a[seat].items()}
            total = sum(result.values())
            value = math.exp(total)
            value_dict[seat] = value
        for seat in value_dict:
            exp_value = math.exp(value_dict[seat])
            choose_prob[seat] = exp_value / sum(math.exp(value) for value in value_dict.values())
        # print("mnl: ", choose_prob)
        return choose_prob

    # Calculate customer choice
    def customer_choice(self, seat_type):
        total_sum = 0
        choose_prob = self.mnl()
        min_key = min(self.seat_price, key=self.seat_price.get)
        p0 = self.seat_price[min_key]
        for seat in self.seat_set:
            pj = self.seat_price['Y'] # 暫時寫這樣
            pj1 = self.seat_price['M'] # 暫時寫這樣
            # print('p0: ',p0 , 'p1: ', pj, 'pj1: ', pj1)
            sum_of_set = (math.exp(-self.alpha*(pj-p0))-math.exp(-self.alpha*(pj1-p0))) * choose_prob[seat_type] 
            # print('sum of set: ', sum_of_set)
            total_sum += sum_of_set
        return total_sum

    # Calculate demand
    def formulation(self, RD2, RD1):
        dm = self.dm()
        # print('dm',dm)
        booking_curve = self.booking_curve(RD2, RD1)
        # print('booking curve: ',booking_curve)
        BR_dict = {}
        
        for seat in self.seat_set:
            # print("calculate", seat, " ing...")
            customer_choice = self.customer_choice(seat)
            # print('customer choice', customer_choice)
            BR = dm * booking_curve * customer_choice
            BR_dict[seat] = BR
            # print(' seat: ', seat, ' predicted demand model from ', RD2,' to ', RD1, 'is', BR)
        print("total demand: ", sum(BR_dict.values()))
        return BR_dict
    
    # Plot demand result
    def plot_demand(self):
        
        # Store calculated result
        every_rd = {seat: [] for seat in model.seat_set}  
        cumulative_demand = {seat: [] for seat in model.seat_set}  
        total_demand = {seat: 0 for seat in model.seat_set}  
        cumulative_total_demand_per_rd = []  # Store each RD cumulative demand

        # Calculate all RD demand
        cumulative_total_demand = 0  # Initialize cumulative demand
        for i in range(2, max_rd+1):
            # print('i: ', i)
            BR_results = model.formulation(i, i-1)  
            total_demand_rd = sum(BR_results.values())  # Calculate demand form all rd
            cumulative_total_demand += total_demand_rd  # update cumulative total deamnd
            cumulative_total_demand_per_rd.append(cumulative_total_demand)  # append cumulative total demand
            for seat, demand in BR_results.items():  
                every_rd[seat].append(demand) 
                total_demand[seat] += demand  
                cumulative_demand[seat].append(total_demand[seat])  
        
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, max_rd), cumulative_total_demand_per_rd, label='Total Demand')
        plt.xlabel('RD')
        plt.ylabel('Total Demand')
        plt.title('Total Demand Model')
        plt.legend()
        plt.xticks(range(0, max_rd, int(max_rd/10)))
        plt.grid(True)
        plt.show()

In [6]:
class AgentActionSpace:
    def __init__(self):
        self.action_list = [0, 1, 2, 3, 4, 5, 6, 7]
        self.n = len(action_list)  # 動作的數量

    def sample(self):
        return np.random.choice(self.action_list)

    def contains(self, action):
        return action in self.action_list

# 創建一個列表動作空間
action_list = [0, 1, 2, 3, 4, 5, 6, 7]
action_space = AgentActionSpace()
print(type(action_space))

<class '__main__.AgentActionSpace'>


In [7]:
'''Uniform distribution'''
class uniform_distribution:

    # Initialize parameters
    def __init__(self, total_demand, total_lambda):
        self.total_demand = total_demand
        self.total_lambda = total_lambda
        self.num_period = int(total_demand/total_lambda)
        self.prob = {'Bus1': 0.1, 'Bus2': 0.2, 'Leis1': 0.2, 'Leis2': 0.2, 'Leis3': 0.3}
    
    # Calculate lambda for each type of customer
    def calculate_lambda(self):
        lambda_list = []
        lambda_list.append(round(1-self.total_lambda, 2))
        for customer in self.prob:
            lambda_list.append(round(self.prob[customer] * self.total_lambda, 2)) 
        print("lambda_list: ", lambda_list)
        return lambda_list

In [8]:
'''Establish Customer class'''
class Customer:

    ''' Initialization '''
    def __init__(self, total_demand, total_lambda):
        self.customer_type = {0: 'f', 1: 'Bus1', 2:'Bus2', 3:'Leis1', 4: 'Leis2', 5: 'Leis3'}
        self.num_customer_type = len(self.customer_type)
        self.customer_preference = {
            'f':{'Y': False, 'M': False, 'K': False},
            'Bus1': {'Y': True, 'M': False, 'K': False},
            'Bus2': {'Y': True, 'M': True, 'K': False},
            'Leis1': {'Y': False, 'M': True, 'K': False},
            'Leis2': {'Y': False, 'M': True, 'K': True},
            'Leis3': {'Y': False, 'M': False, 'K': True},
        }

        # inherent from uniform distribution class
        self.uniform_distribution = uniform_distribution(total_demand, total_lambda)
        self.arrival_rates = self.uniform_distribution.calculate_lambda() # calculate arrival rates for each customer type

        # inherent from aircraft class
        self.aircraft = aircraft()
        self.seat_price = self.aircraft.seat_price

    '''Customer generation'''
    def generate_customer(self):
        random_number = np.random.rand() # generate random number
        probabilities = self.arrival_rates # arrival rates list
        cumulative_probability = 0 # Use cumulative probability decide customer type
        customer_index = 0
        for probability in probabilities:
            cumulative_probability += probability
            if random_number <= cumulative_probability:
                break
            customer_index += 1
        customer_type = self.customer_type[customer_index] # return customer will buy what kind of seat   
        print(f"random_number: {random_number}, customer_index: {customer_index}, customer_type: {customer_type}")
        return customer_type

    '''customer's preference seat under control'''
    def preference_seat(self, customer_type, seat_open):
        preferred_seats = []  
        preferences = self.customer_preference[customer_type]  
        for seat_type, preference in preferences.items():
            if preference and seat_type in seat_open:
                preferred_seats.append(seat_type)
        # print("preference seats: ", preferred_seats)
        return preferred_seats  
    
    '''customer make decision'''
    def make_decision(self, customer_type, seat_open):
        preferred_seats = self.preference_seat(customer_type, seat_open)  
        if preferred_seats:
            cheapest_seat = min(preferred_seats, key=lambda x: self.seat_price[x])
            return cheapest_seat
        else:
            return 'f'

In [9]:
'''Establish env with Single Cabin mulitiple fare classes'''
class AirlineEnvironmentv2:
    
    '''Initialize env parameters'''
    def __init__(self, name):
        
        # name of env
        self.name = name
        
        # inherent aircraft
        self.aircraft = aircraft()
        self.seat_capacity = self.aircraft.seat_capacity # seat limitation
        self.seat_type = self.aircraft.seat_type # seat type
        
        # environment parameters
        self.seat_remain = self.seat_capacity # seat limitation
        self.state = (self.seat_capacity, 1) # Initialize state : (num seat sold, period)
        self.a_s_ref = {0: 'f', 1: 'Y', 2: 'M', 3: 'K', 4: 'YM', 5: 'YK', 6: 'MK', 7: 'YMK'} # action reference to seat open combination
        self.seat_combination = {
            'f': {'f': 0.0, 'Y': 0.0, 'M': 0.0, 'K': 0.0},
            'Y': {'f': 0.0, 'Y': 800, 'M': 0.0, 'K': 0.0},
            'M': {'f': 0.0, 'Y': 0.0, 'M': 500, 'K': 0.0},
            'K': {'f': 0.0, 'Y': 0.0, 'M': 0.0, 'K': 450},
            'YM': {'f': 0.0, 'Y': 800, 'M': 500, 'K': 0.0},
            'YK': {'f': 0.0, 'Y': 800, 'M': 0.0, 'K': 450},
            'MK': {'f': 0.0, 'Y': 0.0, 'M': 500, 'K': 450},
            'YMK': {'f': 0.0, 'Y': 800, 'M': 500, 'K': 450}
        } # Seat combination & seat open
        
        # inherent action space class
        self.action_space = AgentActionSpace() 
        
        # inherent attributes from demand model
        self.max_rd = 20 # total selling RDs
        self.demand_model = demandmodel()
        self.total_demand = sum(self.demand_model.formulation(self.max_rd, 1).values()) # total demand from demand model
        self.total_lambda = 0.8 # total lambda from test
        self.max_period = self.total_demand/self.total_lambda # total period
        
        # inherent attributes from uniform distribution class
        self.uniform_distribution = uniform_distribution(self.total_demand, self.total_lambda)
        self.arrival_rates = self.uniform_distribution.calculate_lambda() # calculate arrival rates for each customer type

        # inherent attributes from Customer class
        self.customer = Customer(self.total_demand, self.total_lambda) 
        
    '''reset env'''
    def reset(self):
        self.seat_remain = self.seat_capacity # Initialize total seat 
        self.state = (self.seat_remain, 1)  # Initialize state
        return self.state

    '''Step'''
    def step_v2(self, state, action):

        # total reward this RD
        total_reward = 0
        
        # Agent choose a seat combination
        seat_open = self.a_s_ref[action]   
        # print("seat_open: ", seat_open)

        # calculate reward this period
        for period in range(1, int(self.max_period/self.max_rd)):
            
            # With remaining seat
            if self.seat_remain > 0:
                
                # Customer generation
                customer_type = self.customer.generate_customer()
                # print("customer type: ", customer_type)
    
                # Customer choose seat
                chosen_seat = self.customer.make_decision(customer_type, seat_open) 
                # print("chosen seat: ", chosen_seat)
    
                # Decide immediate revenue
                reward = self.aircraft.seat_price[chosen_seat] 
    
                # Update seat remain
                if reward > 0:
                    self.seat_remain = self.seat_remain-1
            
            # Without remaining seat
            else:
                # print("No remaining seat.")
                reward = 0

            # add reward to total reward this period
            # print("reward: ", reward)
            total_reward += reward
            # print("total reward: ", total_reward)
    
        # Update rd
        next_rd = state[0, 1].item() + 1
            
        # Check departure or not 
        departure = (next_rd >= self.max_rd+1)
    
        # update state
        state[0, 0] = self.seat_remain
        state[0, 1] = next_rd            
        # print(f"state[0, 0]: {state[0, 0]}, state[0, 1]: {state[0, 1]}")    
        return state.cpu().detach().numpy(), total_reward, departure

In [10]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Define tuple structure, use tuple structure to store info
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

''' Establish Memory Replay Buffer '''
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [12]:
''' 建構 DQN '''
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [13]:
'''Establish environment from Airline env'''
env = AirlineEnvironmentv2("Single route parallel flight.v0")

''' Reinforcement Learning Hyparameter '''
BATCH_SIZE = 128  # Batch size from experience replay buffer
GAMMA = 0.99  # Reward discount rate Gamma
EPS_START = 0.9  # Epilson-Greedy strategy's hyparameter eplison
EPS_END = 0.05 # Final value of eplison 
EPS_DECAY = 1000 # Control eplison decay，higher represent slower decay
TAU = 0.005 # Update rate of target network
LR = 1e-4 # Learning rate of optimizer

''' Simulated Environment Parameter '''
n_actions = env.action_space.n # num action
state = env.reset() # initialize env
n_observations = len(state) # num state

''' Establish Policy Network, Target Network '''
policy_net = DQN(n_observations, n_actions).to(device) # Policy Network
target_net = DQN(n_observations, n_actions).to(device) # Target Network
target_net.load_state_dict(policy_net.state_dict())
memory = ReplayMemory(10000) # Memory size of Replay Buffer
optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True) # AdamW optimizer

''' Agent Select Action'''
steps_done = 0
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        # Exploit：Select Max Value action
        with torch.no_grad():
            # t.max(1) return max expected value column
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        # Explore：Randomly select action
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

total demand:  154.78080374954658
lambda_list:  [0.2, 0.08, 0.16, 0.16, 0.16, 0.24]
lambda_list:  [0.2, 0.08, 0.16, 0.16, 0.16, 0.24]


In [14]:
''' Optimize Network '''
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions)) # Converts batch-array of Transitions to Transition of batch-arrays.

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [15]:
# Initialize Training parameters
num_episodes = 3000
DQN_total_revenues = []
DQN_cumulative_average = []
DQN_per_50_avg = []

# Start Training
for episode in range(num_episodes):
    
    # Initialize environment
    print("---------- Episode ", episode, "-----------")
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    total_revenue = 0 # Initialize total revenue
    
    for step in count():
        
        # print("------- step " ,step + 1 ," ---------")
        
        # agent 選擇動作
        action = select_action(state)
        # print("action: ", action)

        observation, reward, departure = env.step_v2(state, action.item())
        # print("observation: ", observation, "reward: ",  reward, "departure: ", departure)
        reward = torch.tensor([reward], device=device)
        done = departure

        if departure:
            break
        else:
            observation = observation.flatten()
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Update total revenue
        total_revenue += reward

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move on to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
    
    # 將每個 episode 的 total reward 加入列表中  
    DQN_total_revenues.append(total_revenue.cpu().numpy()) 
    
    # 計算每50個 episode 的平均總獎勵
    if episode % 50 == 0 and episode != 0:
        DQN_per_50_avg.append(np.mean(DQN_total_revenues[-50:]))
    
    # 累積平均獎勵
    DQN_cumulative_avg = sum(DQN_total_revenues) / (episode + 1)
    DQN_cumulative_average.append(DQN_cumulative_avg)
    print(f"Episode: {episode}, Total Revenue: {total_revenue}, Cumulative average: {DQN_cumulative_avg}")

---------- Episode  0 -----------
random_number: 0.0505566286272604, customer_index: 0, customer_type: f
random_number: 0.6206713659819961, customer_index: 4, customer_type: Leis2
random_number: 0.7040437645875989, customer_index: 4, customer_type: Leis2
random_number: 0.1821638819019169, customer_index: 0, customer_type: f
random_number: 0.5694514759640049, customer_index: 3, customer_type: Leis1
random_number: 0.1537483793606116, customer_index: 0, customer_type: f
random_number: 0.06274221770833588, customer_index: 0, customer_type: f
random_number: 0.07412673969574624, customer_index: 0, customer_type: f
random_number: 0.9805094220671061, customer_index: 5, customer_type: Leis3
random_number: 0.7955806254122563, customer_index: 5, customer_type: Leis3
random_number: 0.9866635547884887, customer_index: 5, customer_type: Leis3
random_number: 0.5809042441422412, customer_index: 3, customer_type: Leis1
random_number: 0.12798547218575496, customer_index: 0, customer_type: f
random_numbe

KeyboardInterrupt: 

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))  
plt.plot(range(num_episodes), DQN_total_revenues, label='DQN each episode', alpha=0.5, color='b')
plt.plot(range(50, num_episodes, 50), DQN_per_50_avg, label='DQN per 50 episode', alpha=0.5, color='green')
plt.plot(range(num_episodes), DQN_cumulative_average, label='DQN Cumulative')
plt.axhline(y=dp, color='r', label='DP')
plt.axhline(y=fcfs, color='orange', label='FCFS')
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Total Reward per Episode')
plt.legend()  
plt.grid(True)
plt.show()